In [43]:
%load_ext autoreload
%autoreload 2

from util_0701 import *
# from myalgorithm_0801_1 import algorithm

from collections import Counter
import pickle

import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot
import numpy as np
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 실험 내용

- 라이더의 조건과 출발지, 도착지 특징이 변경되었을 때 최적의 가중치 조합이 어떻게 달라지는지 확인하기

### 데이터셋별 라이더 특징 비교

In [36]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

rider_infos = []
rider_fixed_cost_infos = []
K_infos = []
for problem_file in problem_files:
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    rider_info = []
    rider_fixed_cost_info = []

    for v in ALL_RIDERS:
        rider_info.append(v)
        rider_fixed_cost_info.append(v.fixed_cost)
        
    rider_infos.append(rider_info)
    rider_fixed_cost_infos.append(rider_fixed_cost_info)

    K_infos.append(K)

> rider_info 확인 결과 가용한 수량과 고정 비용을 제외하면 모든 테스트 케이스에서 동일함. 그러므로 고정 비용의 변화로 인한 최적 가중치의 변화 결과를 확인하는게 가장 효과적으로 보임

In [23]:
# for v in rider_fixed_cost_infos:
#     print(v)

In [60]:
rider_fixed_cost_comb_st = set([tuple(v) for v in rider_fixed_cost_infos])

print(rider_fixed_cost_comb_st)

{(6000, 6000, 6000), (5000, 5000, 5000), (9000, 4000, 5000), (8000, 8000, 6000)}


### 라이더의 조건과 출발지, 도착지 특징이 변경되었을 때 최적의 가중치 조합이 어떻게 달라지는지 확인

In [70]:

problem_file = r'C:/Users/hsh80/Desktop/LG CNS/stage1_problems/STAGE1_1.json'
# problem_file = '../alg_test_problems_20240429/TEST_K50_2.json'

test_case_name = problem_file.split('/')[-1].split('.')[0]
print(test_case_name)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

for v in ALL_RIDERS:
    print(v)

print(K)

STAGE1_1
Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 20])
Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30])
Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 100])
100


In [71]:
rider_fixed_cost_str = ' '.join([str(rider.fixed_cost) for rider in ALL_RIDERS])
rider_fixed_cost_str

'8000 8000 6000'

In [72]:
for rider_cost_comb in rider_fixed_cost_comb_st:
    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]
    for i in range(3):
        ALL_RIDERS[i].fixed_cost = rider_cost_comb[i]
    rider_fixed_cost_str = ' '.join([str(rider.fixed_cost) for rider in ALL_RIDERS])

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    weight_experiment_data = []
    weight1_grid = [0, 0.5, 1, 1.5, 2]
    weight2_grid = [-3, -2.5, -2, -1.5, -1, -0.5, 0]

    print(test_case_name, rider_cost_comb)
    
    for weight1 in weight1_grid:
        print(weight1, end=' ')
        for weight2 in weight2_grid:
            bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2)
            weight_experiment_data.append((weight1, weight2, cost))
    print()

    with open(f'alg_test_0801_cost_by_weights/{test_case_name}-{rider_fixed_cost_str}.pickle', 'wb') as file:
        pickle.dump(weight_experiment_data, file)

STAGE1_1 (6000, 6000, 6000)
0 0.5 1 1.5 2 
STAGE1_1 (5000, 5000, 5000)
0 0.5 1 1.5 2 
STAGE1_1 (9000, 4000, 5000)
0 0.5 1 1.5 2 
STAGE1_1 (8000, 8000, 6000)
0 0.5 1 1.5 2 


In [ ]:
for rider_cost_comb in rider_fixed_cost_comb_st:
    rider_fixed_cost_str = ' '.join(map(str, rider_cost_comb))
    
    with open(f'alg_test_0801_cost_by_weights/{test_case_name}-{rider_fixed_cost_str}.pickle', 'rb') as file:
        weight_experiment_data = pickle.load(file)

    weight1_grid = [0, 0.5, 1, 1.5, 2]
    weight2_grid = [-3, -2.5, -2, -1.5, -1, -0.5, 0]

    weight_experiment_data.sort()

    weight_experiment_data2 = []
    labels = []
    temp_labels = []
    temp_data = []
    for _, _, v in weight_experiment_data:
        temp_data.append(v)
        temp_labels.append(str(int(v / 10)))

        if len(temp_labels) == len(weight2_grid):
            weight_experiment_data2.append(temp_data)
            labels.append(temp_labels)

            temp_data = []
            temp_labels = []

    weight_experiment_data2.reverse()
    labels.reverse()

    data_df = pd.DataFrame(data=weight_experiment_data2, columns=weight2_grid, index=reversed(weight1_grid))
    plt.figure(figsize=(12, 11))

    plt.title(f'{test_case_name}\n\n{rider_fixed_cost_str}\n')

    ax = sns.heatmap(data_df, cmap="inferno", annot=labels, annot_kws={'fontsize': 16}, fmt='s')
    ax.set_xlabel('Weight 2')
    ax.set_ylabel('Weight 1')

    # plt.show()
    plt.savefig(f'alg_test_0801_cost_by_weights_visualized/{test_case_name}-{rider_fixed_cost_str}.png', bbox_inches='tight')

> 고정 비용이 모두 5000이나 6000일 때는 비슷하나, 그 외는 달라지는 경우가 많았다.